In [84]:
import pandas as pd
dataset = pd.read_csv('hate_speech.csv')
dataset.head(10)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
5,6,0,[2/2] huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...
7,8,0,the next school year is the year for exams.ð...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...


In [5]:
dataset.shape

(5242, 3)

In [7]:
dataset.label.value_counts

<bound method IndexOpsMixin.value_counts of 0       0
1       0
2       0
3       0
4       0
       ..
5237    1
5238    1
5239    1
5240    1
5241    1
Name: label, Length: 5242, dtype: int64>

In [9]:
for index, tweet in enumerate(dataset["tweet"][10:15]):
    print(index+1,"-",tweet)

1 -  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 - we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 - i get to see my daddy today!!   #80days #gettingfed
4 - ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 - i am thankful for having a paner. #thankful #positive     


In [92]:
import re 
 
def clean_text(text): 

    text = re.sub(r'[^a-zA-Z\']', '', text) 
    
    text = re.sub(r'[^\x00-\x7F]+', '', text) 
     
    text = text.lower() 
    return text

In [94]:
dataset['clean_text'] = dataset.tweet.apply(lambda x:clean_text(x))

In [96]:
from nltk.corpus import stopwords 

len(stopwords.words('english')) 
stop = stopwords.words('english') 
 


In [98]:
def gen_freq(text): 
    word_list = [] 
    for tw_words in text.split(): 
        word_list.extend(tw_words)  
        word_freq = pd.Series(word_list).value_counts() 
        word_freq = word_freq.drop(stop, errors='ignore') 
        return word_freq

In [100]:
def any_neg(words): 
    for word in words: 
        if word in ['n', 'no', 'non', 'not'] or re.search(r"\wn't", word): 
            return 1 
        else: 
            return 0

In [102]:
def any_rare(words, rare_100): 
    for word in words: 
        if word in rare_100: 
            return 1 
    else: 
        return 0 
#Check whether prompt words are present 


In [104]:
def is_question(words): 
    for word in words: 
        if word in ['when', 'what', 'how', 'why', 'who', 'where']: 
            return 1 
        else: 
            return 0

In [106]:
word_freq = gen_freq(dataset.clean_text.str) 

rare_100 = word_freq[-100:] 

dataset['word_count'] = dataset.clean_text.str.split().apply(lambda x: len(x)) 

dataset['any_neg'] = dataset.clean_text.str.split().apply(lambda x: any_neg(x)) 

dataset['is_question'] = dataset.clean_text.str.split().apply(lambda x: is_question(x)) 

dataset['any_rare'] = dataset.clean_text.str.split().apply(lambda x: any_rare(x, rare_100)) 

dataset['char_count'] = dataset.clean_text.apply(lambda x: len(x))

In [108]:
from sklearn.model_selection import train_test_split 
X = dataset[['word_count', 'any_neg', 'any_rare', 'char_count', 'is_question']] 
y = dataset.label 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [110]:
from sklearn.naive_bayes import GaussianNB 

model = GaussianNB() 
 
model = model.fit(X_train, y_train) 
 
pred = model.predict(X_test)

In [112]:
model.predict(X_test[5:10])

array([1, 1, 1, 1, 1], dtype=int64)

In [142]:
from sklearn.metrics import accuracy_score
print("Accuracy",accuracy_score(y_test,pred)*100,"%")

Accuracy 42.89799809342231 %


In [140]:
from sklearn.ensemble import RandomForestClassifier

clf_rf=RandomForestClassifier()
clf_rf.fit(X_train, y_train)

rf_pred = clf_rf.predict(X_train).astype(int)

In [144]:
print("y_test shape:", y_test.shape)
print("rf_pred shape:", rf_pred.shape)


y_test shape: (1049,)
rf_pred shape: (4193,)


In [146]:
rf_pred = rf_pred[:len(y_test)]  # Ensure equal length


In [148]:
print("y_test type:", type(y_test))
print("rf_pred type:", type(rf_pred))


y_test type: <class 'pandas.core.series.Series'>
rf_pred type: <class 'numpy.ndarray'>


In [150]:
import numpy as np
y_test = np.array(y_test)
rf_pred = np.array(rf_pred)


In [178]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score 


y_test = np.array(y_test)
rf_pred = np.array(rf_pred)


if y_test.shape != rf_pred.shape:
    print("Shape mismatch:", y_test.shape, rf_pred.shape)
    rf_pred = rf_pred[:len(y_test)]


if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    y_test = np.argmax(y_test, axis=1)  
if len(rf_pred.shape) > 1 and rf_pred.shape[1] > 1:
    rf_pred = np.argmax(rf_pred, axis=1)


print("Unique y_test:", np.unique(y_test))
print("Unique rf_pred:", np.unique(rf_pred))


print(confusion_matrix(y_test, rf_pred)) 
print(classification_report(y_test, rf_pred)) 
print("Accuracy:", accuracy_score(y_test, rf_pred))


Unique y_test: [0 1]
Unique rf_pred: [0 1]
[[393 206]
 [315 135]]
              precision    recall  f1-score   support

           0       0.56      0.66      0.60       599
           1       0.40      0.30      0.34       450

    accuracy                           0.50      1049
   macro avg       0.48      0.48      0.47      1049
weighted avg       0.49      0.50      0.49      1049

Accuracy: 0.5033365109628217


In [180]:

from sklearn.linear_model import LogisticRegression 
logreg = LogisticRegression (class_weight='balanced') 
logreg.fit(X_train, y_train) 
y_pred = logreg.predict(X_test) #predicting the values 



In [182]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()


              precision    recall  f1-score   support

           0       0.64      0.58      0.61       599
           1       0.50      0.57      0.54       450

    accuracy                           0.58      1049
   macro avg       0.57      0.58      0.57      1049
weighted avg       0.58      0.58      0.58      1049



In [184]:
def tokenizer_porter(text): 
    return [porter.stem(word) for word in text.split()]